# Import Libraries

In [201]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import random
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import catboost

%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# Read Dataset

In [202]:
#Read datasets
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')
#Read test datasets
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

# Fix metric on ceiling_height_agl:m and cloud_base_agl:m

Found that most likely the **ceiling_height_agl:m** and **cloud_base_agl:m** starts with values in meters, and then suddenly switches into using feet at 26.03.2020. Therefore I transform the values before 26.03.2020 into feet. 

In [203]:
mask_a = X_train_observed_a['date_forecast'] < '2020-03-26'
mask_b = X_train_observed_b['date_forecast'] < '2020-03-26'
mask_c = X_train_observed_c['date_forecast'] < '2020-03-26'

# Apply the conversion from meters to feet (1 meter = 3.28084 feet)
conversion_factor = 3.28084

X_train_observed_a.loc[mask_a, 'ceiling_height_agl:m'] *= conversion_factor
X_train_observed_b.loc[mask_b, 'ceiling_height_agl:m'] *= conversion_factor
X_train_observed_c.loc[mask_c, 'ceiling_height_agl:m'] *= conversion_factor

X_train_observed_a.loc[mask_a, 'cloud_base_agl:m'] *= conversion_factor
X_train_observed_b.loc[mask_b, 'cloud_base_agl:m'] *= conversion_factor
X_train_observed_c.loc[mask_c, 'cloud_base_agl:m'] *= conversion_factor

As the features have a 0.83 correlation, I choose to go forward with ceiling height

# Concat data

In [204]:

#add location to each sample
train_a["location"] = "A"
train_b["location"] = "B"
train_c["location"] = "C"
X_train_estimated_a["location"] = "A"
X_train_estimated_b["location"] = "B"
X_train_estimated_c["location"] = "C"
X_train_observed_a["location"] = "A"
X_train_observed_b["location"] = "B"
X_train_observed_c["location"] = "C"
X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

#copy 23:45 value to 00:00 as it isn't there
def fill_last(frame):
    copy = frame.copy()
    
    copy["date_forecast"] = copy["date_forecast"] + pd.Timedelta(minutes=15)
    
    copy = copy[copy["date_forecast"].apply(lambda time : time.hour == 0 and time.minute == 0)]
    
    frame = pd.concat([
        frame,
        copy
    ])
    return frame.drop_duplicates(subset="date_forecast", keep="first")

#fill last
X_test_estimated_a = fill_last(X_test_estimated_a)
X_test_estimated_b = fill_last(X_test_estimated_b)
X_test_estimated_c = fill_last(X_test_estimated_c)

#remove extra minute 00 sample
X_train_observed_a = X_train_observed_a.iloc[:-1,:]
X_train_observed_b = X_train_observed_b.iloc[:-1,:]
X_train_observed_c = X_train_observed_c.iloc[:-1,:]

#add date_calc column same as date_forecast column to observed data
X_train_observed_a.insert(0, "date_calc", X_train_observed_a["date_forecast"])
X_train_observed_b.insert(0, "date_calc", X_train_observed_b["date_forecast"])
X_train_observed_c.insert(0, "date_calc", X_train_observed_c["date_forecast"])

#concat all the samples and remove date_calc column
X_train_raw = pd.concat([X_train_observed_a,
                     X_train_observed_b,
                     X_train_observed_c,
                     X_train_estimated_a,
                     X_train_estimated_b,
                     X_train_estimated_c,
                     X_test_estimated_a,
                     X_test_estimated_b,
                     X_test_estimated_c])


## Clean and preprocess data

In [205]:

#map snow density to one and zero
X_train_raw["snow_density:kgm3"] = X_train_raw["snow_density:kgm3"].apply(lambda a : np.isnan(a)).map({True: 0, False: 1})

#fix ceiling_height NaN values to -666 because the docs hints to it
#also rename the features with their proper metric and remove the old ones
X_train_raw["ceiling_height_agl:ft"] = X_train_raw["ceiling_height_agl:m"].fillna(-666)
X_train_raw["cloud_base_agl:ft"] = X_train_raw["cloud_base_agl:m"].fillna(-666)
X_train_raw.drop(columns=["ceiling_height_agl:m", "cloud_base_agl:m"], inplace=True)

#categorizing ceiling_height_agl:ft
#found categories on google
X_train_raw['ceiling_height_agl:ft'] = pd.cut(X_train_raw['ceiling_height_agl:ft'], bins=[float('-inf'), 0, 500, 1000, 3000, 5000, 12000, float('inf')], labels=[-666, 1, 2, 3, 4, 5, 6])

#decided to drop cloud_base_agl:ft because of high correlation to ceiling_height_agl:ft
X_train_raw.drop(columns=["cloud_base_agl:ft"], inplace=True)

#categorizing dew_or_rime:idx
dew_or_rime_categories = {
    -1.0: "Rime",
    0.0: "None",
    1.0: "Dew"
}
X_train_raw['dew_or_rime:idx'] = X_train_raw['dew_or_rime:idx'].map(dew_or_rime_categories)

#casting floats to int for categorical features
X_train_raw['is_day:idx'] = X_train_raw['is_day:idx'].astype(int)

#remove some weird artifacts from train_b target values
train_b = pd.concat([train_b[:18690], train_b[20142:]])
train_b["rolling"] = train_b["pv_measurement"].rolling(4).mean()
train_b["keep"] = train_b["pv_measurement"] - train_b["rolling"] != 0 + train_b["pv_measurement"].apply(lambda a: a==0)
train_b = train_b[train_b["keep"]]
train_b = train_b.iloc[:,:3]

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

train_a["id"] = -10
train_b["id"] = -10
train_c["id"] = -10

X_test_targets = X_test_targets.rename(columns = {"prediction" : "pv_measurement"})

targets = pd.concat([train_a,
                     train_b,
                     train_c,
                     X_test_targets]).dropna()


## Split into timeframes

In [206]:

features00 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 0)].copy()
features00["merge_time"] = features00["date_forecast"]


features15 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 15)].copy()
features15["merge_time"] = features15["date_forecast"] + pd.Timedelta(minutes=-15)


features30 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 30)].copy()
features30["merge_time"] = features30["date_forecast"] + pd.Timedelta(minutes=-30)


features45 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 45)].copy()
features45["merge_time"] = features45["date_forecast"] + pd.Timedelta(minutes=-45)


X_train_raw["date_forecast"] = X_train_raw["date_forecast"] + pd.Timedelta(minutes = -60)
features60 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 00)].copy()
features60["merge_time"] = features60["date_forecast"]

dataset = targets
dataset = dataset.rename(columns={"time": "merge_time"})

display(X_train_raw)

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location,ceiling_height_agl:ft
0,2019-06-02 22:00:00,2019-06-02 21:00:00,7.7,1.230,0.0,0.0,None,280.299988,0.0,0.0,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,0,0.0,0.0,-0.0,0.1,342.834015,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,A,5
1,2019-06-02 22:15:00,2019-06-02 21:15:00,7.7,1.229,0.0,0.0,None,280.299988,0.0,0.0,0.0,0.0,99.000000,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.500000,0.0,0.0,993.900024,999.900024,0.0,0.0,72.199997,1006.000000,0,0.0,0.0,-0.0,0.2,346.294006,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,A,5
2,2019-06-02 22:30:00,2019-06-02 21:30:00,7.7,1.228,0.0,0.0,None,280.299988,0.0,0.0,0.0,0.0,99.199997,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.099976,0.0,0.0,993.599976,999.599976,0.0,0.0,71.199997,1005.599976,0,0.0,0.0,-0.0,0.2,349.768005,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,A,5
3,2019-06-02 22:45:00,2019-06-02 21:45:00,7.7,1.226,0.0,0.0,None,280.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1005.799988,0.0,0.0,993.299988,999.299988,0.0,0.0,70.199997,1005.299988,0,0.0,0.0,-0.0,0.2,353.251007,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,A,5
4,2019-06-02 23:00:00,2019-06-02 22:00:00,7.7,1.225,0.0,0.0,None,280.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,0,0.0,0.0,-0.0,0.2,356.742004,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,A,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,2023-06-18 07:00:05,2023-06-19 23:00:00,10.7,1.193,0.0,0.0,None,285.600006,0.0,0.0,0.0,0.0,27.799999,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1011.000000,0.0,0.0,995.299988,1001.200012,0.0,0.0,59.200001,1007.000000,0,0.0,0.0,-0.0,0.0,6.356000,-3.011,0.0,293.700012,43.500000,46996.800781,4.0,-2.0,3.5,-0.0,C,5
2303,2023-06-21 07:00:30,2023-06-22 23:00:00,9.4,1.228,0.0,0.0,None,283.299988,0.0,0.0,0.0,0.0,95.599998,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1013.900024,0.0,0.0,997.799988,1003.799988,0.0,0.0,83.199997,1009.900024,0,0.0,0.0,-0.0,0.0,6.206000,-2.996,0.0,285.399994,96.800003,33542.898438,1.3,-0.6,1.1,-0.0,C,3
2399,2023-06-25 07:01:23,2023-06-26 23:00:00,9.8,1.187,0.0,0.0,None,284.200012,0.0,0.0,0.0,0.0,2.000000,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1004.799988,0.0,0.0,989.200012,995.000000,0.0,0.0,53.099998,1000.799988,0,0.0,0.0,-0.0,0.0,6.015000,-3.071,0.0,295.500000,2.000000,48980.699219,3.1,-2.0,2.5,-0.0,C,-666
2687,2023-06-29 07:00:05,2023-06-30 23:00:00,9.3,1.220,0.0,0.0,None,283.100006,0.0,0.0,0.0,0.0,100.000000,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1004.000000,0.0,0.0,989.200012,995.200012,0.0,0.1,88.800003,1001.200012,0,0.0,0.0,-0.0,0.2,5.837000,-3.254,0.1,284.399994,100.000000,9935.700195,1.8,1.6,-0.9,0.0,C,3


# Define helper functions

In [207]:
#averages the features meassured at target time +00, +15, +30, +45 and +60
def add_feature_average_00_60(dataset, f00, f15, f30, f45, f60, column_name, categorical=False):
    dataset = pd.merge(
        left=dataset,
        right = f00[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner")
    dataset = pd.merge(
        left=dataset,
        right = f15[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_15"])
    dataset = pd.merge(
        left=dataset,
        right = f30[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_30"])
    dataset = pd.merge(
        left=dataset,
        right = f45[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_45"])
    dataset = pd.merge(
        left=dataset,
        right = f60[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_60"])


    dataset[column_name] = (dataset[column_name] +
                            dataset[column_name + "_15"] +
                            dataset[column_name + "_30"] +
                            dataset[column_name + "_45"] +
                            dataset[column_name + "_60"])/5
    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)
    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

#adds a single feature from one observation
def add_feature(dataset, f, column_name, count=True, categorical=False):
    if count:
        if categorical:
            add_to_categorical_features(column_name)
        else:
            add_to_numerical_features(column_name)
    return pd.merge(
            left=dataset,
            right=f[["location", "merge_time", column_name]],
            on=["location", "merge_time"],
            how="inner"
  )

#adds an One Hot Encoding of the column to the dataset
def OHE(dataset, f, column_name, suffix=""):

    dataset = pd.merge(
        left=dataset,
        right = f[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner")

    values = dataset[column_name].unique()

    for value in values:
        dataset[column_name + "_" + suffix + str(value)] = dataset[column_name].apply(lambda a : a == value).map({True: 1, False: 0})

    dataset = dataset.drop([column_name], axis=1)
    return dataset

def OHE_all(dataset, f00, f15, f30, f45, f60, column_name):
    dataset = OHE(dataset, f00, column_name, suffix="00_")
    dataset = OHE(dataset, f15, column_name, suffix="15_")
    dataset = OHE(dataset, f30, column_name, suffix="30_")
    dataset = OHE(dataset, f45, column_name, suffix="45_")
    dataset = OHE(dataset, f60, column_name, suffix="60_")

    return dataset

#adds all observations
def add_all(dataset, f00, f15, f30, f45, f60, column_name, categorical=False):
    dataset[column_name + "_00"] = add_feature(dataset, f00, column_name, count=False,categorical=categorical)[column_name]
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False, categorical=categorical)[column_name]

    feature_names = [column_name + "_00", column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]
    for feat in feature_names:
        if categorical:
            add_to_categorical_features(feat)
        else:
            add_to_numerical_features(feat)

    return dataset

#finds mode (typetall) for a row
def find_mode_with_priority(row, priority_list, max=False):
    # Check for prioritized values in the row and set it as the mode if it is
    for value in priority_list:
        if value in row.values:
            return value

    # Calculate the mode for the row
    if max:
        mode_value = row.mode().max()
    else:
        mode_value = row.mode().min()

    return mode_value

def add_most_frequent_feature(dataset, f15, f30, f45, f60, column_name, priority_list, max=False, categorical=True):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False)[column_name]

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    dataset[column_name] = dataset[feature_names].apply(find_mode_with_priority, args=(priority_list, max), axis=1)

    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)

    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

def add_accumulated(dataset, f15, f30, f45, f60, column_name, time_interval, categorical=False):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False)[column_name]

    time_multiplier = 15/time_interval

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    dataset[column_name] = dataset[feature_names].sum(axis=1)*time_multiplier

    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)
    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

def add_accumulated_all(dataset, f15, f30, f45, f60, column_name, time_interval, categorical=False):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name)[column_name]

    time_multiplier = 15/time_interval

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    for feat in feature_names:
        dataset[feat] = dataset[feat]*time_multiplier
    if categorical:
        add_to_categorical_features(column_name)    
    else:
        add_to_numerical_features(column_name)

    return dataset

numerical_feature_names = []
categorical_feature_names = []

def add_to_numerical_features(feature_name):
    numerical_feature_names.append(feature_name)

def add_to_categorical_features(feature_name):
    categorical_feature_names.append(feature_name)


# Feature info

### Features removed because of correlations:

- **fresh_snow_12h:cm**: fresh_snow_24h:cm = **0.82**

- **fresh_snow_3h:cm**: fresh_snow_1h:cm = **0.81**

- **fresh_snow_6h:cm**: fresh_snow_24h:cm = **0.83**

- **diffuse_rad:W**: diffuse_rad_1h:J = **0.99**

- **direct_rad:W**: direct_rad_1h:J = **0.99**

- **pressure_100m:hPa**: msl_pressure:hPa = **1.00**

- **pressure_50m:hPa**: msl_pressure:hPa = **1.00**

- **sfc_pressure:hPa**: msl_pressure:hPa = **1.00**

- **absolute_humidity_2m:gm3**: t1000:hPa = **0.90**

- **air_density_2m:kgm3**: t1000:hPa = **0.90**

- **dew_point_2m:K**: t1000:hPa = **0.91**

- **clear_sky_rad:W**: sun_elevation = **0.83**

- **clear_sky_energy_1h:J**: sun_elevation = **0.82**

- **total_cloud_cover:p**: effective_cloud_cover:p = **0.94**

- **cloud_base_agl:m**: ceiling_height_agl:m = **0.83** (after recalculating to feet)

- **elevation:m**: highly correlated to location

- **is_in_shadow:idx**: highly correlated to is_day:idx

### High correlations still in the set:

- **is_day:idx**: sun_elevation = **0.81**

- **diffuse_rad_1h:J**: sun_elevation = **0.80**

### Other features removed:

- **snow_drift:idx**: Almost exclusively 0, so doesn't add much data

- **wind_speed_w_1000hPa:ms**: In relation to the dummy data found in the docs this data is really weird (binary as opposed to continous values)


# Add numerical features

In [208]:
# SNOW AND PRECIPITATION

# tar verdi fra +60 siden den viser måling mellom 00 og 60, #!kan det være gunstig å ha med 3h, 6h, 12h????
# dataset = add_feature(dataset, features60, "fresh_snow_1h:cm")
# dataset = add_feature(dataset, features60, "fresh_snow_24h:cm")

# bruker bare 24h fordi den har høyest korrelasjon med pv_measurement
dataset = add_feature(dataset, features60, "fresh_snow_24h:cm")

# tar alle akkumulerte verdier og ganger med 3 for å få en bedre verdi (ikke helt etter boka menmen...)
dataset = add_all(dataset, features00, features15, features30, features45, features60, "precip_5min:mm")
# disse tar jeg bare gjennomsnittet av
#dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "snow_depth:cm")
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "snow_water:kgm2")
# tar akkumulert verdi og ganger med 3/2 for å få en bedre verdi (ikke helt etter boka menmen...)
#dataset = add_accumulated(dataset, features15, features30, features45, features60, "snow_melt_10min:mm", 10)


# ACCUMULATIVE FEATURES

# tar verdi fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "diffuse_rad_1h:J")
# tar verdi fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "direct_rad_1h:J")#!Try without

# PRESSURE

# tar gjennomsnittet da dette er punktmålinger ##kan hende denne burde kjøres per kvarter
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "msl_pressure:hPa")


# TEMPERATURE

# gjennomsnitt siden variasjonen hvert kvarter sannsynligvis er lav? ##kan hende denne burde kjøres per kvarter
# update: kjører hvert kvarter
dataset = add_all(dataset, features00, features15, features30, features45, features60, "t_1000hPa:K")


# SUN

#legger til alle siden har testing har vist at disse er svært viktige
dataset = add_all(dataset, features00, features15, features30, features45, features60, "sun_azimuth:d")
dataset = add_all(dataset, features00, features15, features30, features45, features60, "sun_elevation:d")

#tar verdien fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "clear_sky_energy_1h:J")


# DAY AND SHADOW

#gjennomsnitt fordi jeg vet ikke
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "visibility:m")


# CLOUDS

#gjennomsnitt fordi verdien er trolig momentan
#SANNSYNLIGVIS VELDIG VIKTIG
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "effective_cloud_cover:p")


# HUMIDITY AND RIME

#tar gjennomsnitt fordi jeg vet ikke #!diskuter
#dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "prob_rime:p")
#tar gjennomsnitt fordi jeg vet ikke #!diskuter
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "relative_humidity_1000hPa:p")


# WIND

# Gjennomsnitt fordi lite variabel #! try without
# dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_u_10m:ms")
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_v_10m:ms")
# dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_10m:ms")


# OTHERS (Up for discussion)
# Gjennomsnitt fordi?
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "super_cooled_liquid_water:kgm2")

# Standardize data

In [209]:
# make trig transform on solar angles
def apply_trig(dataset, feature_name, suffixes):
    for suffix in suffixes:
        column_name = f"{feature_name}_{suffix}"
        if column_name in dataset:
            dataset[column_name] = dataset[column_name].apply(lambda d: np.cos((d * np.pi) / 180))

# apply trig transform on solar angles
suffixes = ['00', '15', '30', '45', '60']
apply_trig(dataset, 'sun_azimuth:d', suffixes)
apply_trig(dataset, 'sun_elevation:d', suffixes)

# # all features except pv_measurement, merge_time, location and id
# columns_to_exclude = ['merge_time', 'pv_measurement', 'location', 'id']
# # standardize the features
# scaler = StandardScaler()
# scaled_columns = scaler.fit_transform(dataset.drop(columns=columns_to_exclude))
# scaled_dataset = pd.DataFrame(scaled_columns, columns=dataset.drop(columns=columns_to_exclude).columns)

# # add the excluded columns back to the dataset
# dataset = pd.concat([dataset[columns_to_exclude], scaled_dataset], axis=1)


## Trying first without standardization


# def standardize(dataset, feature_name, minus_min=False):
#     if minus_min:
#         dataset[feature_name] = (dataset[feature_name] - dataset[feature_name].min())/dataset[feature_name].std()
#     else:
#         dataset[feature_name] = dataset[feature_name]/dataset[feature_name].std()
    
#     return dataset

# def standardize_all(dataset, feature_name, accumulated=False, minus_min=False):
#     if not accumulated:
#         dataset[feature_name + "_00"] = standardize(dataset, feature_name + "_00")[feature_name + "_00"]

#     dataset[feature_name + "_15"] = standardize(dataset, feature_name + "_15", minus_min=minus_min)[feature_name + "_15"]
#     dataset[feature_name + "_30"] = standardize(dataset, feature_name + "_30", minus_min=minus_min)[feature_name + "_30"]
#     dataset[feature_name + "_45"] = standardize(dataset, feature_name + "_45", minus_min=minus_min)[feature_name + "_45"]
#     dataset[feature_name + "_60"] = standardize(dataset, feature_name + "_60", minus_min=minus_min)[feature_name + "_60"]

#     return dataset

# # standardize the features
# dataset = standardize(dataset, 'fresh_snow_24h:cm')
# dataset = standardize_all(dataset, 'precip_5min:mm')
# #dataset = standardize(dataset, 'snow_depth:cm')
# dataset = standardize(dataset, 'snow_water:kgm2')
# #dataset = standardize(dataset, 'snow_melt_10min:mm')
# dataset = standardize(dataset, 'diffuse_rad_1h:J')
# dataset = standardize(dataset, 'direct_rad_1h:J')
# dataset = standardize(dataset, 'msl_pressure:hPa', minus_min=True)
# dataset = standardize_all(dataset, 't_1000hPa:K', minus_min=True)
# dataset = standardize_all(dataset, 'sun_azimuth:d')
# dataset = standardize_all(dataset, 'sun_elevation:d')
# dataset = standardize(dataset, 'clear_sky_energy_1h:J')
# dataset = standardize(dataset, 'visibility:m')
# dataset = standardize(dataset, 'effective_cloud_cover:p')
# # dataset = standardize(dataset, 'prob_rime:p')
# dataset = standardize(dataset, 'relative_humidity_1000hPa:p')
# # dataset = standardize(dataset, 'wind_speed_u_10m:ms')
# dataset = standardize(dataset, 'wind_speed_v_10m:ms')
# # dataset = standardize(dataset, 'wind_speed_10m:ms')
# dataset = standardize(dataset, 'super_cooled_liquid_water:kgm2')


# Add categorical features

In [210]:
# reason I'm adding it here is to prevent the massive -666 value getting in the way of the scaling
# add most frequent bc don't wanna have 100 000 features, also don't add priority list and pick the min value if conflict
dataset = add_most_frequent_feature(dataset, features00, features15, features30, features45, "ceiling_height_agl:ft", priority_list=[], max=False)
#! test this aswell
# dataset = add_all(dataset, features00, features15, features30, features45, features60, "ceiling_height_agl:ft")

# add most frequent bc don't wanna have 100 000 features
dataset = add_most_frequent_feature(dataset, features15, features30, features45, features60, "precip_type_5min:idx", priority_list=[3,4,2], max=True)
#! test this aswell
# dataset = add_all(dataset, features00, features15, features30, features45, features60, "precip_type_5min:idx")

# categorizing ceiling_height_agl:ft
# found categories on google
labels=["VFR1", "LIFR", "IFR", "MVFR", "VFR4", "VFR3", "VFR2"]
ceiling_height_agl_ft_categories = {
    -666: "VFR1",
    1: "LIFR",
    2: "IFR",
    3: "MVFR",
    4: "VFR4",
    5: "VFR3",
    6: "VFR2",
}
# map the values to their labels
dataset['ceiling_height_agl:ft'] = dataset['ceiling_height_agl:ft'].map(ceiling_height_agl_ft_categories)

#categorizing precip_type_5min:idx
#found this in the docs
precip_types = {
    0: "None",
    1: "Rain",
    2: "Rain_and_snow_mixed",
    3: "Snow",
    4: "Sleet",
    5: "Freezing_rain",
    6: "Hail",
}
#map the values to their labels
dataset['precip_type_5min:idx'] = dataset['precip_type_5min:idx'].map(precip_types)
    
# Add feature from 60 because pretty consistent
dataset = add_feature(dataset, features60, "snow_density:kgm3", categorical=True)

# OHE av kategorisk variabel #!Opp til diskusjon om man skal ta gjennomsnitt eller flere av målingene
dataset = add_all(dataset, features00, features15, features30, features45, features60, "dew_or_rime:idx", categorical=True)
# Husk at denne er mappet til strings lenger opp

# tar alle verdiene siden disse nok er ekstremt viktige for modellen og gir ikke mening å standardisere
dataset = add_all(dataset, features00, features15, features30, features45, features60, "is_day:idx", categorical=True)

#mapper location til tall og legger til som kategorisk variabel
# location_mapping = {
#     "A": 1,
#     "B": 2,
#     "C": 3
# }
# dataset["location"] = dataset["location"].map(location_mapping)
add_to_categorical_features("location")

# 4 precip_5min:mm values are NaN, set them to 0 because precip_type is 0.0 (None)
# 3 precip_type:idx values are NaN, set them to 0.0 (None)
dataset = dataset.fillna(0.0)

In [211]:
display(dataset.head())
print(dataset.drop(columns=['merge_time', 'pv_measurement']).shape)
print(len(numerical_feature_names))
print(len(categorical_feature_names))
print(len(numerical_feature_names) + len(categorical_feature_names))

,merge_time,pv_measurement,location,id,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,ceiling_height_agl:ft,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60
0,2019-06-02 22:00:00,0.00,A,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.18,0.000000,0.000000,1006.140015,285.899994,286.100006,286.299988,286.600006,286.799988,0.955454,0.971524,0.984097,0.993071,0.998384,0.998439,0.997972,0.997566,0.997254,0.997062,0.000000,40649.164062,99.180000,71.179993,-0.40,0.0,VFR3,None,0,None,None,None,None,None,0,0,0,0,0
1,2019-06-02 23:00:00,0.00,A,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.000000,0.000000,1005.079956,286.799988,286.899994,286.899994,287.000000,287.000000,0.998384,0.999992,0.997884,0.992075,0.985583,0.997062,0.997002,0.997081,0.997290,0.997672,0.000000,31111.119141,99.799995,67.599998,0.36,0.0,VFR3,None,0,None,None,None,None,None,0,0,0,0,0
2,2019-06-03 00:00:00,0.00,A,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.46,0.000000,0.000000,1004.500000,287.000000,287.000000,286.899994,286.899994,286.899994,0.985583,0.973531,0.957968,0.939004,0.916774,0.997672,0.998054,0.998497,0.998962,0.999399,0.000000,11297.320312,100.000000,68.580002,0.76,0.0,VFR3,None,0,None,None,None,None,None,0,0,0,0,0
3,2019-06-03 01:00:00,0.00,A,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.50,7743.299805,0.000000,1003.900024,286.899994,286.799988,286.700012,286.600006,286.500000,0.916774,0.891418,0.863096,0.831973,0.798215,0.999399,0.999755,0.999967,0.999972,0.999701,6546.899902,2393.800049,100.000000,74.800003,0.90,0.0,VFR4,None,0,None,None,None,None,None,0,0,0,1,1
4,2019-06-03 02:00:00,19.36,A,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.22,60137.601562,3158.300049,1003.000000,286.500000,286.500000,286.399994,286.399994,286.399994,0.798215,0.762002,0.723485,0.682840,0.640204,0.999701,0.999084,0.998051,0.996532,0.994462,102225.898438,14631.379883,79.659996,80.419998,0.92,0.0,VFR4,None,0,None,None,None,None,None,1,1,1,1,1


(90962, 46)
31
14
45


In [212]:
dataset.isna().sum()

merge_time                        0
pv_measurement                    0
location                          0
id                                0
fresh_snow_24h:cm                 0
precip_5min:mm_00                 0
precip_5min:mm_15                 0
precip_5min:mm_30                 0
precip_5min:mm_45                 0
precip_5min:mm_60                 0
snow_water:kgm2                   0
diffuse_rad_1h:J                  0
direct_rad_1h:J                   0
msl_pressure:hPa                  0
t_1000hPa:K_00                    0
t_1000hPa:K_15                    0
t_1000hPa:K_30                    0
t_1000hPa:K_45                    0
t_1000hPa:K_60                    0
sun_azimuth:d_00                  0
sun_azimuth:d_15                  0
sun_azimuth:d_30                  0
sun_azimuth:d_45                  0
sun_azimuth:d_60                  0
sun_elevation:d_00                0
sun_elevation:d_15                0
sun_elevation:d_30                0
sun_elevation:d_45          

# Split into training set and test set

In [213]:
testset = dataset[dataset["id"].apply(lambda id: id != -10)]
testset = testset.drop(columns=["merge_time", "id", "pv_measurement", 'location'])

In [214]:
dataset = dataset[dataset["id"].apply(lambda id: id == -10)]
dataset = dataset.drop("id", axis=1)

dataset = dataset.sort_values(by="merge_time")

trainingsetX = dataset.iloc[:int(len(dataset)*0.8)]
trainingsetY = trainingsetX["pv_measurement"]
evaluationsetX = dataset.iloc[int(len(dataset)*0.8):]
evaluationsetY = evaluationsetX["pv_measurement"]

# drop the target values from the training and evaluation sets
trainingsetX = trainingsetX.drop(columns=["pv_measurement", "merge_time"])
evaluationsetX = evaluationsetX.drop(columns=["pv_measurement", "merge_time"])

display(trainingsetX)
display(evaluationsetX)
display(trainingsetY)
display(evaluationsetY)

,location,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,ceiling_height_agl:ft,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60
34059,B,0.0,0.00,0.00,0.00,0.00,0.05,0.06,0.000000,0.000000,986.859985,278.399994,278.299988,278.200012,278.100006,278.000000,0.974429,0.949939,0.918177,0.879922,0.836047,0.654213,0.660159,0.667949,0.677415,0.688393,0.000000,16017.700195,85.080002,89.059998,2.66,0.00,VFR4,Rain,0,None,None,None,None,None,0,0,0,0,0
34060,B,0.0,0.00,0.06,0.06,0.06,0.00,0.10,0.000000,0.000000,986.099976,278.000000,278.100006,278.100006,278.100006,278.200012,0.836047,0.787430,0.734950,0.679403,0.621517,0.688393,0.700710,0.714143,0.728503,0.743577,0.000000,12777.299805,97.239990,88.259995,3.22,0.00,VFR4,None,0,None,None,None,None,None,0,0,0,0,0
34061,B,0.0,0.06,0.00,0.00,0.00,0.00,0.10,0.000000,0.000000,985.979980,278.200012,278.100006,278.000000,277.899994,277.799988,0.621517,0.561925,0.501178,0.439720,0.377922,0.743577,0.759180,0.775121,0.791213,0.807300,0.000000,10592.379883,96.580002,87.199997,3.46,0.12,VFR4,None,0,None,None,None,None,None,0,0,0,0,0
34062,B,0.0,0.00,0.00,0.00,0.00,0.06,0.10,0.000000,0.000000,986.299988,277.799988,277.700012,277.500000,277.399994,277.200012,0.377922,0.316096,0.254467,0.193241,0.132550,0.807300,0.823235,0.838870,0.854096,0.868787,0.000000,6259.920410,96.919998,88.919998,3.44,0.20,VFR4,Rain,0,None,None,None,None,None,0,0,0,0,0
34063,B,0.0,0.00,0.00,0.00,0.00,0.00,0.10,0.000000,0.000000,986.860046,277.200012,277.500000,277.799988,278.100006,278.399994,0.132550,0.072507,0.013229,-0.045241,-0.102827,0.868787,0.882874,0.896254,0.908880,0.920696,0.000000,6074.040039,96.440002,91.360001,1.84,0.16,VFR4,None,0,None,None,None,None,None,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85015,C,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,1035.040039,277.899994,277.799988,277.700012,277.700012,277.600006,0.339526,0.282442,0.224781,0.166700,0.108312,0.986309,0.990529,0.994030,0.996761,0.998679,0.000000,37034.460938,0.000000,52.539997,0.90,0.00,VFR1,None,0,Rime,Rime,Rime,Rime,Rime,0,0,0,0,0
24534,A,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,1035.099976,277.700012,277.600006,277.600006,277.500000,277.399994,0.339575,0.282475,0.224798,0.166700,0.108312,0.986349,0.990557,0.994049,0.996772,0.998684,0.000000,25896.500000,29.799999,53.460003,0.64,0.00,LIFR,None,0,Rime,Rime,Rime,Rime,Rime,0,0,0,0,0
24535,A,0.0,0.00,0.00,0.00,0.00,0.00,0.00,39226.601562,17824.300781,1035.320068,277.399994,277.399994,277.299988,277.299988,277.200012,0.108312,0.049704,-0.008988,-0.067667,-0.126234,0.998684,0.999757,0.999974,0.999330,0.997836,29177.400391,26199.218750,29.799999,54.299999,0.72,0.00,LIFR,None,0,Rime,Rime,None,None,None,0,0,1,1,1
85016,C,0.0,0.00,0.00,0.00,0.00,0.00,0.00,38037.199219,0.000000,1035.280029,277.600006,277.600006,277.500000,277.500000,277.399994,0.108312,0.049739,-0.008936,-0.067597,-0.126130,0.998679,0.999756,0.999974,0.999330,0.997831,29194.900391,37151.320312,0.360000,53.260002,1.10,0.00,VFR1,None,0,Rime,Rime,Rime,Rime,Rime,0,0,1,1,1


,location,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,ceiling_height_agl:ft,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60
85017,C,0.0,0.0,0.00,0.00,0.00,0.0,0.00,148292.00000,113965.00000,1035.599976,277.399994,277.299988,277.299988,277.200012,277.100006,-0.126130,-0.184466,-0.242464,-0.300007,-0.356950,0.997831,0.995501,0.992371,0.988489,0.983910,2.624277e+05,31796.259766,1.100000,51.799999,0.96,0.00,VFR1,None,0,Rime,Rime,None,None,None,1,1,1,1,1
85018,C,0.0,0.0,0.00,0.00,0.00,0.0,0.00,269865.50000,345338.09375,1035.980103,277.100006,277.000000,276.799988,276.700012,276.600006,-0.356950,-0.413136,-0.468408,-0.522528,-0.575319,0.983910,0.978703,0.972950,0.966739,0.960167,6.530747e+05,27831.980469,5.700000,50.299999,0.20,0.00,VFR1,None,0,None,None,None,None,None,1,1,1,1,1
24537,A,0.0,0.0,0.00,0.00,0.00,0.0,0.00,294024.90625,281121.59375,1036.040039,276.899994,276.799988,276.700012,276.500000,276.399994,-0.357097,-0.413295,-0.468578,-0.522707,-0.575491,0.983951,0.978757,0.973018,0.966823,0.960269,6.519056e+05,17827.259766,33.739998,50.999996,-0.26,0.00,LIFR,None,0,None,None,None,None,None,1,1,1,1,1
85019,C,0.0,0.0,0.00,0.00,0.00,0.0,0.00,348962.50000,604286.37500,1036.179932,276.600006,276.500000,276.500000,276.399994,276.299988,-0.575319,-0.626495,-0.675783,-0.722895,-0.767479,0.960167,0.953344,0.946379,0.939388,0.932494,1.045562e+06,26384.980469,14.900000,52.900002,-0.50,0.00,VFR1,None,0,None,None,None,None,None,1,1,1,1,1
24538,A,0.0,0.0,0.00,0.00,0.00,0.0,0.00,386585.90625,516511.59375,1036.219971,276.399994,276.399994,276.299988,276.299988,276.200012,-0.575491,-0.626658,-0.675950,-0.723039,-0.767624,0.960269,0.953465,0.946520,0.939549,0.932677,1.043731e+06,16565.900391,34.559998,52.599998,-0.70,0.00,LIFR,None,0,None,None,None,None,None,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,A,0.0,0.0,0.00,0.00,0.00,0.0,0.04,0.00000,0.00000,1014.380005,274.399994,274.399994,274.299988,274.200012,274.200012,0.950250,0.968257,0.982340,0.992375,0.998275,0.982839,0.981286,0.980029,0.979113,0.978567,0.000000e+00,13311.120117,98.220001,79.199997,2.52,0.10,MVFR,None,0,None,None,None,None,None,0,0,0,0,0
88800,C,0.0,0.0,0.00,0.00,0.00,0.0,0.08,0.00000,0.00000,1014.479980,274.299988,274.200012,274.200012,274.200012,274.100006,0.950223,0.968235,0.982323,0.992366,0.998271,0.982723,0.981161,0.979897,0.978978,0.978426,0.000000e+00,11463.541016,98.699997,79.599998,2.28,0.14,MVFR,None,0,None,None,None,None,None,0,0,0,0,0
34058,A,0.0,0.0,0.05,0.05,0.05,0.0,0.04,0.00000,0.00000,1014.020020,274.200012,274.200012,274.200012,274.299988,273.799988,0.998275,0.999981,0.997478,0.990790,0.983191,0.978567,0.978408,0.978638,0.979258,0.981006,0.000000e+00,15539.981445,94.880005,79.940002,2.72,0.06,MVFR,None,0,None,None,None,None,None,0,0,0,0,0
62731,B,0.0,0.0,0.00,0.00,0.00,0.0,0.04,0.00000,0.00000,1014.079956,274.200012,274.200012,274.299988,274.299988,273.799988,0.998276,0.999981,0.997478,0.990790,0.983188,0.978567,0.978408,0.978642,0.979258,0.981009,0.000000e+00,15873.179688,94.880005,79.940002,2.74,0.06,MVFR,None,0,None,None,None,None,None,0,0,0,0,0


34059       0.00
34060       0.00
34061       0.00
34062       0.00
34063       0.00
          ...   
85015       0.00
24534       0.00
24535     155.54
85016       9.80
24536    1137.18
Name: pv_measurement, Length: 71041, dtype: float64

85017      49.00
85018     156.80
24537    2403.28
85019     245.00
24538    3380.08
          ...   
34057       0.00
88800      -0.00
34058       0.00
62731      -0.00
88801      -0.00
Name: pv_measurement, Length: 17761, dtype: float64

# Making model

In [215]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Get column indices
categorical_indices = [trainingsetX.columns.get_loc(col) for col in categorical_feature_names]

# # X_train and y_train are your feature matrix and target variable
# train_data = Pool(data=trainingsetX, label=trainingsetY, cat_features=categorical_indices)

try:
    train_data = Pool(
        data=trainingsetX,
        label=trainingsetY,
        cat_features=categorical_indices
    )
except Exception as e:
    # Print the error message
    print("CatBoostError:", e)
    
    # Extract the column name from the error message (this is just an example, adjust as needed)
    # Note: This assumes that the error message contains information about the feature causing the error.
    error_message = str(e)
    start_idx = error_message.find("feature_idx=")
    end_idx = error_message.find("]", start_idx)
    feature_idx_str = error_message[start_idx:end_idx]
    feature_idx = int(feature_idx_str.split("=")[1])
    
    # Get the column name using the feature index
    column_name = trainingsetX.columns[feature_idx]
    
    print(f"The error occurred in the column: {column_name}")

# Specify hyperparameters
model = CatBoostClassifier(iterations=500,  # Number of trees (boosting rounds)
                           depth=10,        # Depth of the tree
                           learning_rate=0.05,  # Step size for gradient descent
                           cat_features=categorical_indices)  # Indices of categorical features

# Train the model
model.fit(train_data)

# Assuming X_test is your test set
predictions = model.predict(evaluationsetX)

# Assuming y_test is your true labels
accuracy = accuracy_score(evaluationsetY, predictions)
print(f'Accuracy: {accuracy}')

# Additional evaluation metrics
print(classification_report(evaluationsetY, predictions))
print(confusion_matrix(evaluationsetY, predictions))